##### `If your files originate from a Thermo instrument:`

Copy all your *.raw files to data/raw directory and you can use the following script to generate a samples.tsv file automatically:

In [ ]:
!(cd data/raw && ls *.raw > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list, index_col= None)
df["sample_name"]=df["sample_name"].replace(".raw*", value="", regex=True)
df["comment"] = " "
df["MAPnumber"] = " "
df.to_csv("config/samples.tsv", sep="\t")
df

##### `If your files originate from another instrument:`

Copy all your already converted files *.mzML to data/mzML directory and you can use the following script to generate a samples.tsv file automatically:

In [ ]:
!(cd data/mzML && ls *.mzML > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/mzML/filelist.txt", names=header_list, index_col= None)
df["sample_name"]=df["sample_name"].replace(".mzML*", value="", regex=True)
df["comment"] = " "
df["MAPnumber"] = " "
df.to_csv("config/samples.tsv", sep="\t")
df

##### `Create a GNPS metadata table:`
This is datafile-dependent so it is preferable to do it interactively through a Jupyter notebook

In [1]:
!(cd results/Interim/mzML && ls PCpeak*.mzML > filelist.txt)

In [3]:
import pandas as pd
import numpy as np 
import os 

path= "results/GNPSexport/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

import pandas as pd
header_list = ["filename"]
df=pd.read_csv("results/Interim/mzML/filelist.txt", names=header_list, index_col= None)
df["filename"]= "results/Interim/mzML/"+ df["filename"]
df['ATTRIBUTE_MAPID'] = np.arange(len(df))
df["ATTRIBUTE_MAPID"]= "MAP" + df["ATTRIBUTE_MAPID"].astype(str)
df['ATTRIBUTE_genomeID']=df['filename'].str.extract(r'(NBC_?\d*)')
df['ATTRIBUTE_genomeIDMDNA']=df['filename'].str.extract(r'(MDNAWGS?\d*|MDNA_WGS_?\d*)')
df['ATTRIBUTE_genomeID']=df['ATTRIBUTE_genomeID'].fillna(df['ATTRIBUTE_genomeIDMDNA'])
df["ATTRIBUTE_media"]= df['filename'].str.extract(r'(ISP2|DNPM|FPY12|MA|soyM\d*)')
df["ATTRIBUTE_comment"]= df['ATTRIBUTE_genomeID'].astype(str) +"_" + df["ATTRIBUTE_media"].astype(str)
df=df.drop(columns="ATTRIBUTE_genomeIDMDNA")
df.to_csv("results/GNPSexport/metadata.tsv", sep='\t')
df

,filename,ATTRIBUTE_MAPID,ATTRIBUTE_genomeID,ATTRIBUTE_media,ATTRIBUTE_comment
0,PCpeak_20210607_UMETAB211_POS_00029_DNPM_HLB70...,MAP0,NBC_00029,DNPM,NBC_00029_DNPM
1,PCpeak_20210607_UMETAB211_POS_00029_ISP2_HLB70...,MAP1,NBC_00029,ISP2,NBC_00029_ISP2
2,PCpeak_20210607_UMETAB211_POS_00029_MA_HLB70_P...,MAP2,NBC_00029,MA,NBC_00029_MA
3,PCpeak_20210607_UMETAB211_POS_00029_soyM_HLB70...,MAP3,NBC_00029,soyM,NBC_00029_soyM
4,PCpeak_20210607_UMETAB211_POS_00105_DNPM_HLB70...,MAP4,NBC_00105,DNPM,NBC_00105_DNPM
5,PCpeak_20210607_UMETAB211_POS_00105_ISP2_HLB70...,MAP5,NBC_00105,ISP2,NBC_00105_ISP2
6,PCpeak_20210607_UMETAB211_POS_00105_MA_HLB70_P...,MAP6,NBC_00105,MA,NBC_00105_MA
7,PCpeak_20210607_UMETAB211_POS_00105_soyM_HLB70...,MAP7,NBC_00105,soyM,NBC_00105_soyM
8,PCpeak_20210608_UMETAB211_POS_00207_DNPM_HLB70...,MAP8,NBC_00207,DNPM,NBC_00207_DNPM
9,PCpeak_20210608_UMETAB211_POS_00207_ISP2_HLB70...,MAP9,NBC_00207,ISP2,NBC_00207_ISP2


In [5]:
!(cd results/Interim/mzML && rm filelist.txt)